<a href="https://colab.research.google.com/github/lymoelopez/automated-filipino-fake-news-detector/blob/main/evidenceCollector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evidence Collector

### I. Web Searcher

In [1]:
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install google

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###### Web Search Filter

In [3]:
URL_banList = ["facebook", "twitter", "youtube", "pdf", "blog", "tiktok", "instagram", "youtu.be", "mp4", "mp3", "audiobook", "podcast", "spotify"]

def URLfilter(url):
  lowercasedURL = url.lower()
  if all(excludedURL not in lowercasedURL for excludedURL in URL_banList):
    return url

###### Google Search Api

In [9]:
from googlesearch import search

def webSearcher(claim):

  filteredList = []
  
  while len(filteredList) <=10:
    if len(filteredList) >= 10:
      break
    for url in search(claim, lang="tl", num=10, pause=3):
      filteredURL = URLfilter(url)
      if filteredURL and (filteredURL not in filteredList):
        filteredList.append(filteredURL)
      if len(filteredList) >= 10:
        break

    return filteredList

In [10]:
sampleClaim = "may banta sa buhay ni Teves"

urlList = webSearcher(sampleClaim)

In [11]:
# Number of obntained links
print(len(urlList))

10


In [12]:
# Top 10 URLs
print(urlList)

['https://www.philstar.com/pilipino-star-ngayon/bansa/2023/03/23/2253752/pangulong-marcos-walang-banta-sa-buhay-ni-teves', 'https://www.philstar.com/pilipino-star-ngayon/bansa/2023/03/14/2251478/teves-may-banta-sa-buhay-kaya-takot-umuwi-legal-counsel', 'https://net25.com/article/13821', 'https://www.remate.ph/walang-banta-sa-buhay-ni-teves-pbbm/', 'https://news.abs-cbn.com/news/03/22/23/banta-sa-buhay-teves-nagpaliwanag-sa-di-pagsipot-sa-kamara', 'https://dzar1026.ph/banta-sa-buhay-ni-cong-arnie-teves-inaalam-ng-militar/', 'https://radyopilipinas.ph/rp1/banta-sa-buhay-ni-rep-teves-kapwa-itinanggi-ng-pnp-at-afp/', 'https://dzme1530.ph/cong-arnie-teves-hindi-pa-dapat-ituring-na-pugante-atty-topacio/', 'https://balita.net.ph/2023/03/22/mag-landing-ka-kung-saan-mo-gusto-marcos-umapela-ulit-kay-teves-na-umuwi-na/', 'https://pilipinomirror.com/banta-kay-teves-tinitingnan-ng-pnp/']


### II. Evidence Selector

###### Article Extractor

In [8]:
!pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 72.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=91afa80b568614c2268171be39b13046e87a73ca3013101e4eb8015b49a9221e
  Stored in directory: /root/.cache/pip/wheels/94/ad/df/a2a01300cea47d5695f242f7e925a805970106fd9e4b151468
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3356 sha256=fcfcfd264066628ca919c3426942fde8bd12eb4a191c0458ef0d8

In [13]:
import newspaper

def articleExtractor(url):
  article = newspaper.Article(url)
  article.download()
  article.parse()
  return article.text

In [14]:
sampleUrl = "https://www.philstar.com/pilipino-star-ngayon/bansa/2023/03/23/2253752/pangulong-marcos-walang-banta-sa-buhay-ni-teves"

print(articleExtractor(sampleUrl))

Pangulong Marcos: Walang banta sa buhay ni Teves

President Ferdinand "Bongbong" Marcos Jr. graces the 72nd anniversary celebration of the Department of Social Welfare and Development (DSWD) at its central office in Batasan Road, Quezon City on January 31, 2023.

MANILA, Philippines — Dahil wala namang banta sa buhay kaya pinapauwi na ni Pangulong Ferdinand Marcos Jr. si Negros Oriental Congressman Arnie Teves.

Si Teves ay sinasabing sangkot sa pagpatay kay Governor Roel Degamo nitong Marso 4.

Sa ambush interview kay Pangulong Marcos sa ika-126 Founding Anniversary ng Philippine Army sa Fort Bonifacio sa Taguig City, sinabi nito na mayaman naman si Teves at mayroong private jet.

Kaya bahala na si Teves na mamili kung saang airbase gustong mag-landing at papalibutan na lang ito ng mga sundalo at walang makakalapit sa kanya ng isang kilometro para lang masiguro ang kanyang kaligtasan.

Matatandaan na sinabi ni Teves na may banta sa kanyang seguridad kung kaya nag-aalangan siyang umuwi

In [18]:
# extracted articles from top 10 URLs

for url in urlList:
  print(url)
  print(articleExtractor(url))
  print("\n")

https://www.philstar.com/pilipino-star-ngayon/bansa/2023/03/23/2253752/pangulong-marcos-walang-banta-sa-buhay-ni-teves
Pangulong Marcos: Walang banta sa buhay ni Teves

President Ferdinand "Bongbong" Marcos Jr. graces the 72nd anniversary celebration of the Department of Social Welfare and Development (DSWD) at its central office in Batasan Road, Quezon City on January 31, 2023.

MANILA, Philippines — Dahil wala namang banta sa buhay kaya pinapauwi na ni Pangulong Ferdinand Marcos Jr. si Negros Oriental Congressman Arnie Teves.

Si Teves ay sinasabing sangkot sa pagpatay kay Governor Roel Degamo nitong Marso 4.

Sa ambush interview kay Pangulong Marcos sa ika-126 Founding Anniversary ng Philippine Army sa Fort Bonifacio sa Taguig City, sinabi nito na mayaman naman si Teves at mayroong private jet.

Kaya bahala na si Teves na mamili kung saang airbase gustong mag-landing at papalibutan na lang ito ng mga sundalo at walang makakalapit sa kanya ng isang kilometro para lang masiguro ang ka

###### Cosine Similarity

# References

[1] https://www.geeksforgeeks.org/performing-google-search-using-python-code/